<a href="https://colab.research.google.com/github/dipanshuhaldar/Custom_NER_SpaCy/blob/master/NER_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Build a Custom NER Model with SpaCy

In [0]:
#install SpaCy
!pip install -U spacy
!python -m spacy download en

In [0]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/entity_annotated_corpus')

In [0]:
import numpy as np
import pandas as pd

In [0]:
ner_data = pd.read_csv('ner_dataset.csv',engine = 'python')
ner_data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [0]:
row_num = ner_data.loc[ner_data['Sentence #']=='Sentence: 261'].index.values[0] - 1
ner_data = ner_data.head(row_num)
ner_data.tail()

,Sentence #,Word,POS,Tag
5717,NaN,from,IN,O
5718,NaN,the,DT,O
5719,NaN,oil-for-food,NN,O
5720,NaN,program,NN,O
5721,NaN,itself,PRP,O


In [0]:
#deleting Sentence # column 
del ner_data['Sentence #']
del ner_data['POS']
ner_data.columns

Index(['Word', 'Tag'], dtype='object')

In [0]:
indexNames = ner_data[ner_data['Word'] == '"'].index
# Delete these row indexes from dataFrame
ner_data.drop(indexNames , inplace=True)

#indexNames = ner_data[ner_data['Word'] == '10,000'].index
# Delete these row indexes from dataFrame
#ner_data.drop(indexNames , inplace=True)
  

In [0]:
ner_data.to_csv('ner_corpus.tsv', sep = '\t', header = False, index=False)

In [0]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("ner_corpus.tsv",'ner_corpus_260.json','abc')

In [0]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

input_file = 'ner_corpus_260.json'
output_file = 'ner_corpus_260_custom'

training_data = []
lines=[]
with open(input_file, 'r') as f:
  lines = f.readlines()
for line in lines:
  data = json.loads(line)
  text = data['content']
  entities = []  

for line in lines:
  data = json.loads(line)
  text = data['content']
  entities = [] 
  for annotation in data['annotation']:
    point = annotation['points'][0]
    labels = annotation['label']
    if not isinstance(labels, list):
      labels = [labels] 

    for label in labels:
      entities.append((point['start'], point['end'] + 1 ,label))  

  training_data.append((text, {"entities" : entities}))

print(training_data) 

with open(output_file, 'wb') as fp:
  pickle.dump(training_data, fp)

[('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country ', {'entities': [(48, 54, 'B-geo'), (77, 81, 'B-geo'), (111, 118, 'B-gpe')]}), ('Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as Bush Number One Terrorist and Stop the Bombings ', {'entities': [(107, 111, 'B-per')]}), ('They marched from the Houses of Parliament to a rally in Hyde Park ', {'entities': [(57, 61, 'B-geo'), (62, 66, 'I-geo')]}), ('Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 ', {'entities': []}), ("The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton ", {'entities': [(57, 64, 'B-geo'), (129, 137, 'B-geo'), (75, 80, 'B-org'), (81, 86, 'I-org'), (103, 110, 'B-gpe')]}), ("The party is divided over Britain 's participation in the Iraq conflict and the

In [0]:
# Training additional entity types using spaCy
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

LABEL = ['I-geo', 'B-geo', 'I-art', 'B-art', 'B-tim', 'B-nat', 'B-eve', 'O', 'I-per', 'I-tim', 'I-nat', 'I-eve', 'B-per', 'I-org', 'B-gpe', 'B-org', 'I-gpe']

"""
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon
"""

# Loading training data 
with open ('ner_corpus_260_custom', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

model= 'en'
new_model_name= 'new_model'
output_dir= '/content/drive/My Drive/entity_annotated_corpus'
n_iter= 1000  

"""Setting up the pipeline and entity recognizer, and training the new entity."""

if model is not None:
  nlp = spacy.load(model)  # load existing spacy model
  print("Loaded model '%s'" % model)
else:
  nlp = spacy.blank('en')  # create blank Language class
  print("Created blank 'en' model")

if 'ner' not in nlp.pipe_names:
  ner = nlp.create_pipe('ner')
  nlp.add_pipe(ner)
else:
  ner = nlp.get_pipe('ner')

for i in LABEL:
        ner.add_label(i)   # Add new entity labels to entity recognizer

if model is None:
  optimizer = nlp.begin_training()
else:
  optimizer = nlp.entity.create_optimizer()

# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(n_iter):
    random.shuffle(TRAIN_DATA)
    losses = {}
    batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35,losses=losses)
    print('Losses', losses) 

# Test the trained model
test_text = 'Gianni Infantino is the president of FIFA.'
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
  print(ent.label_, ent.text)      


# Save model 
if output_dir is not None:
  output_dir = Path(output_dir)
  if not output_dir.exists():
    output_dir.mkdir()
  nlp.meta['name'] = new_model_name  # rename model
  nlp.to_disk(output_dir)
  print("Saved model to", output_dir)

In [0]:
output_dir= '/content/drive/My Drive/entity_annotated_corpus'

import spacy
from spacy.util import minibatch, compounding

# Test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
#test_text = 'John Lee is the chief of CBSE'
test_text = 'Americans suffered from H5N1'

doc2 = nlp2(test_text)
for ent in doc2.ents:
  print(ent.label_, ent.text)

Loading from /content/drive/My Drive/entity_annotated_corpus
B-gpe Americans
B-nat H5N1
